In [ ]:
# import sys
import pathlib
# import os

# # in order to do relative import
# sys.path.append(str(pathlib.Path(os.getcwd()).parent))
# sys.path.append(str(pathlib.Path(os.getcwd()).parent.parent))

In [ ]:
import numpy as np
import torch

from residual_actions import ResidualActionsSettings, ResidualActionsLearner

# Read data

In [ ]:
with open('datasets/expdata_1_2024_12_03_dodge_and_collect_four_min_4800/states.npy', 'rb') as f:
    states = np.load(f, allow_pickle=True)
with open('datasets/expdata_1_2024_12_03_dodge_and_collect_four_min_4800/actions.npy', 'rb') as f:
    actions = np.load(f, allow_pickle=True)

In [ ]:
# skip close fromes, essentialy changing from 60 fps to 15 fps
states = states[::4, ...]
actions = actions[::4, ...]

In [ ]:
SETTINGS = ResidualActionsSettings(
    history_size=16,
    hidden_channels_memory=128,
    hidden_channels_behavior=256,
    batch_size=32,
    optim_learning_rate=0.001,
    target_loss=1e-07,
    force_stop_at_plateau_epochs=20,
    train_log_frequency=10,
    running_loss_window_size=10,
    min_epochs=100,
    grace_epochs_after_min_epochs=20,
    max_epochs=900
)

# Train

In [ ]:
learner = ResidualActionsLearner(
    state_space_size=states.shape[-1],
    action_space_size=actions.shape[-1],
    settings=SETTINGS,
    device='cuda'
)

In [ ]:
learner.process_and_train_full(
    states_train=torch.from_numpy(states).to(torch.float32).cuda(),
    actions_train=torch.from_numpy(actions).to(torch.float32).cuda()
)